# Trabalho Prático de Machine Learning
#### Aluna: Cecília Regina Oliveira de Assis - 2019697720
#### Professor: Adriano Veloso

In [ ]:
# Importacoes
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import model_selection 
from sklearn import naive_bayes 
from sklearn import neighbors
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import metrics

from IPython.display import display

In [ ]:
# Caminho do arquivo de entrada
INPUT_FILEPATH = 'koi_data.csv'

# Variavel de predicao
TARGET = 'koi_disposition'

# Label das features que nao serao utilizadas no treinamento dos modelos
NOT_USED = [TARGET, 'kepoi_name']

# Número de splits da validacao cruzada (cross validation - CV)
K_FOLDS = 5

## Resumo dos Dados

O conjunto de dados usado neste notebook corresponde a características de candidatos a exoplanetas (KOI - Kepler Object of Interest) enumeradas a partir de leituras realizadas pela sonda espacial Kepler. 

A primeira coluna da base de dados identifica o KOI enquanto a segunda a traz a sua classificação (FALSE POSITIVE ou CONFIRMED). As demais colunas são features sobre o KOI extraídas de diversas formas.

Fonte dos dados: [Kepler Exoplanet Search Results](https://www.kaggle.com/nasa/kepler-exoplanet-search-results)

In [ ]:
# Leitura do dataset para um DataFrame do Pandas
# usando separador default (virgula)
df = pd.read_csv(INPUT_FILEPATH)

# Cria uma lista com as features
features = list(df.columns)
for x in NOT_USED:
    features.remove(x)

# Imprime metadados do conjunto
print('Numero de linhas: {}'.format(df.shape[0]))
print('Numero de colunas: {}'.format(df.shape[1]))
print('Dados faltando: {}'.format(df.isnull().sum().sum()))

# Exibe a proporcao de cada classe
print('\nProporcao de cada classe:')
display((df[TARGET].value_counts()/len(df)*100).round(2).to_frame(TARGET +" (%)").T)

# Apresenta um breve sumário do conjunto
print('\nSumario do conjunto de dados:')
display(df[features].describe()) 

print('\nAmostra:')
with pd.option_context('max_columns', 8):
    display(df.head(20))
    
print('Target: {}'.format(TARGET))
print('Features:')
print('\n'.join(['  ' + x for x in features]))

mpl.rc('font', size=14)

## Preprocessamento

### Target

In [ ]:
df[TARGET] = (df[TARGET] == 'CONFIRMED').astype(int)

print('Resultado:')

display(df[[TARGET]].sample(10))

### Normalização

In [ ]:
# Reacria a lista com as features
features = list(df.columns)
for x in NOT_USED:
    features.remove(x)

# Normaliza os atributos usando a metodologia MinMax
#   Define o intervalo de cada feature
minmax = df[features].max() - df[features].min()

#   Subtrai o mínimo valor da feature e divide pelo intervalo 
df[features] = (df[features] - df[features].min()) / minmax

display(df[features].head(20))

## Classificação

### Apresentação gráfica

In [ ]:
# Define as variaveis que armazenam o melhor modelo por metrica
best_ein             = ['', 1.01]
best_eout            = ['', 1.01]
best_precision_false = ['', -1]
best_precision_true  = ['', -1]
best_recall_false    = ['', -1]
best_recall_true     = ['', -1]

# Define uma funcao que atualiza o melhor modelo por metrica
def update_best(title, model):
    if model['train_acc'] < best_ein[1]:
        best_ein[1] = results['train_acc']
        best_ein[0] = title
    
    if model['test_acc'] < best_eout[1]:
        best_eout[1] = model['test_acc']
        best_eout[0] = title
        
    if model['precision'][0] > best_precision_false[1]:
        best_precision_false[1] = model['precision'][0]
        best_precision_false[0] = title
        
    if model['precision'][1] > best_precision_true[1]:
        best_precision_true[1] = model['precision'][1]
        best_precision_true[0] = title
        
    if model['recall'][0] > best_recall_false[1]:
        best_recall_false[1] = model['recall'][0]
        best_recall_false[0] = title
        
    if model['recall'][1] > best_recall_true[1]:
        best_recall_true[1] = model['recall'][1]
        best_recall_true[0] = title

In [ ]:
# Define uma funcao que avalia se o valor passado eh None ou nao
def xstr(s):
    return 'None' if s is None else str(s)

# Define uma funcao que apresenta graficamente o 
# resultado (train_values, test_values) do classificador (title)
# Os valores do hiperparametro e sua definicao eh recebido
# pelas variaveis params e xlabel, respectivamente
def plot_values(title, xlabel, params, results):
    # Inicializa grafico
    fig = plt.figure(figsize=(19,5)) 
    ax  = fig.add_subplot(111)

    # Define titulo do grafico
    fig.suptitle(title, fontsize=20, fontweight='bold')
    
    for idx, value in enumerate(results):
        # Define valores a serem exibidos
        param   = xstr(params[idx])
        v_train = value['train_acc']
        v_test  = value['test_acc']
        
        ax.scatter(param, v_train, color='orange')
        ax.scatter(param, v_test,  color='blue')
        
        # Define o label dos pontos plotados
        ax.annotate('  %.3f' % v_train, xy=(param, v_train))
        ax.annotate('  %.3f' % v_test, xy=(param, v_test))
        
    # Define rotulo dos eixos x e y
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Accurary')
    
    # Exibte o grafico
    plt.show()

In [ ]:
# Define uma funcao que plota as curvas roc de um dado modelo
def plot_roc_curve(title, clf):
    folds = len(train_idx_folds)
        
    # Inicializa grafico
    fig = plt.figure(figsize=(7,7))
    ax  = fig.add_subplot(111)
    
    # Define titulo do grafico
    fig.suptitle('Roc Curve  - {}'.format(title), 
                 fontsize=20, fontweight='bold')
    
    for i in range(folds):
        # Recupera os índices dos conjuntos de treino e teste
        x_train = df.loc[train_idx_folds[i], features]
        y_train = df.loc[train_idx_folds[i], TARGET]
        x_test  = df.loc[test_idx_folds[i],  features]
        y_test  = df.loc[test_idx_folds[i],  TARGET]
        
        # Treina e testa o modelo
        model = clf.fit(x_train, y_train)
        
        # Calcula e armazena a revocacao e a precisao
        y_proba = clf.predict_proba(x_test)[:, 1]
        
        # Calcula as taxas de falso positivo e negativo para a curva roc
        fpr, tpr, _ = metrics.roc_curve(y_test, y_proba)

        # Define valores a serem exibidos
        ax.plot(fpr, tpr, label='Run {}'.format(i + 1))
        ax.plot([0, 1], [0, 1], color='grey', linestyle='--')

        # Configura os limites dos eixos
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    
    # Define rotulo dos eixos
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    
    # Habilita a exibicao e legendas
    ax.legend()
    
    # Exibte o grafico
    plt.show()

### Validação cruzada

In [ ]:
# Cria o gerador de k-folds
kf = model_selection.KFold(n_splits=K_FOLDS)

train_idx_folds = []
test_idx_folds  = []
for train_idx, test_idx in kf.split(df):
    train_idx_folds.append(train_idx)
    test_idx_folds.append(test_idx)

# Imprime breve sumário dos k-folds
folds = len(train_idx_folds)

print('Quantidade de folds (k): {}'.format(folds))
print('Folds:')
print('\tTamanho folds de treino: {}'.format(len(train_idx_folds[0])))
print('\tTamanho folds de teste: {}\n'.format(len(test_idx_folds[0])))
for i in range(folds):
    print('\tTreino: {}'.format(train_idx_folds[i]))
    print('\tTeste:  {}\n'.format(test_idx_folds[i]))


In [ ]:
# Define uma funcao que treina e testa o classificador clf,
# sobre o conjunto de dados df
def run_classifier(df, clf):
    folds      = len(train_idx_folds)

    train_acc = 0
    test_acc  = 0
    precision = []
    recall    = []
    
    # Para cada fold:
    for i in range(folds):
        # Recupera os índices dos conjuntos de treino e teste
        x_train = df.loc[train_idx_folds[i], features]
        y_train = df.loc[train_idx_folds[i], TARGET]
        x_test  = df.loc[test_idx_folds[i],  features]
        y_test  = df.loc[test_idx_folds[i],  TARGET]
        
        # Treina e testa o modelo
        model = clf.fit(x_train, y_train)
        
        # Armazenas acuracias
        train_acc += model.score(x_train, y_train)
        test_acc  += model.score(x_test, y_test)
        
        # Calcula e armazena a revocacao e a precisao
        y_pred = clf.predict(x_test)
        
        p = metrics.precision_score(y_test, y_pred, average=None)
        r = metrics.recall_score(y_test, y_pred, average=None)
        
        precision.append(p) 
        recall.append(r)
        
    # Calcula e armazena as medias de acuracia, precisao e revocacao
    train_acc /= folds
    test_acc  /= folds
    precision  = np.asarray(precision).mean(axis=0)
    recall     = np.asarray(recall).mean(axis=0)
    
    return {
        'train_acc': train_acc, 
        'test_acc': test_acc, 
        'precision': precision, 
        'recall': recall
    }

In [ ]:
# Apresenta o valor das métricas apuradas pelo classificador
def show_metrics(title, clf):
    v_test  = clf['test_acc']
    v_train = clf['train_acc']
    
    print('\t\tMetrics - {}\n'.format(title))    
    print('Erros:')
    print('\tEmpírico: %.3f' % (1 - v_train))
    print('\tEsperado: %.3f' % (1 - v_test))
    print('\tDiferença de aproximação: %.3f' % abs(v_train - v_test))
    
    print('Precisao:')
    print('\tFalse Positive (No):  %.3f' % clf['precision'][0])
    print('\tConfirmed      (Yes): %.3f' % clf['precision'][1])  

    print('Revocacao:')
    print('\tFalse Positive (No):  %.3f' % clf['recall'][0])
    print('\tConfirmed      (Yes): %.3f\n' % clf['recall'][1])  
    
    # Atualiza o melhor modelo para cada metrica
    update_best(title, clf)

### Classificadores

#### Naive-Bayes (baseline)

In [ ]:
title = 'Naive-Bayes'
gnb   = naive_bayes.GaussianNB()

# Treina, testa o modelo e apresenta as metricas apuradas
show_metrics(title, run_classifier(df=df, clf=gnb))

# Apresenta dos resultados
plot_roc_curve(title, gnb)

##### Análise dos resultados

O _baseline_ do experimento foi o algoritmo __Naive-Bayes__, um método probabilístico, não parametrizado, que assume uma distribuição gaussiana dos dados e a independência condicional das variáveis.

Por não definir hiperparâmetros, o classificador apura resultados satisfatórios com quantidade pequenas de dados. Tal situação ocorreu no presente experimento, tendo o algoritmo apresentado uma boa aproximação entre erro empírico ($E_{in}$) e erro esperado ($E_{out}$).

#### Decision Tree

In [ ]:
# Define os valores a serem testados no hiperparametro
values  = [None, 2, 5, 10, 20, 30, 40, 50, int(df.shape[0]/100), 100]
results = []

# Testa cada valor no hiperparametro e apresenta as metricas apuradas
for value in values:
    title = 'Decision Tree (maxdepth={})'.format(value)
    dtree = tree.DecisionTreeClassifier(max_depth=value)
    dtree = run_classifier(df=df, clf=dtree)
    show_metrics(title, dtree)
    
    results.append(dtree)

# Demonstra graficamente os resultados da Arvore de Decisao
title  = 'Decision Tree'
xlabel = 'Tree max depth'

# Apresenta dos resultados
plot_values(title, xlabel, values, results)

##### Análise dos resultados

O hiperparâmetro variado para esse algoritmo foi o "max_depth" que define a altura máxima da árvore. Dez valores foram utilizados, sendo um deles correspondente a altura infinita, ou segundo aborda o _scikit learn_, até que todas as folhas sejam puras ou não apresentem mais de 2 exemplos (valor padrão para o hiperparâmetro "min_samples").

Com a variação de tal hiperparâmetro pode-se observar que todos os modelos apresentaram _overfit_ que é o a situação onde o modelo decora os exemplos de treino, não performando bem a generalização. 

Ainda com o _overfitting_ aquele modelo que se sobressaiu dentre os demais, demonstrando melhor aproximação de erro empírico com erro esperado, foi aquele com altura máxima igual a 5.

### SVM

In [ ]:
# Define os valores a serem testados no hiperparametro
values  = ['linear', 'sigmoid', 'poly', 'rbf']
results = []

# Testa cada valor no hiperparametro e apresenta as metricas apuradas
for value in values:
    title  = 'Support Vector Machine (kernel={})'.format(value)
    s = svm.SVC(gamma='auto', kernel=value)
    s = run_classifier(df=df, clf=s)
    show_metrics(title, s)
    
    results.append(s)

# Demonstra graficamente os resultados do SVM
title  = 'Support Vector Machine'
xlabel = 'Type of kernel'

# Apresentacao dos resultados
plot_values(title, xlabel, values, results)

##### Análise dos resultados

Para o algoritmo __SVM__, quatro tipos de kernel (hiperparâmetro), funções que mudam a representação do dado, foram utilizados.

A escolha de hiperparâmetro que se saiu melhor foi aquela que adotou o kernel "linear". Em contrapartida, o kernel "polinomial", que representa as features como polinômios, apresentou características de _underfitting_ ao não generalizar suficientemente o modelo tanto para os dados de treino quanto para os teste. 

É importante notar que o kernel "polinomial" nos dois primeiros folds de teste predisse a classe False Positive (0 ou No) para todos os exemplos, levantando um aviso da biblioteca quanto ao cálculo das métricas (precisão e revocação). Tal comportamento teve por consequência baixa taxa de precisão do modelo para a classe Confirmed (1 ou Yes). 

Por fim, o hiperparâmetro "C", que define o tradeoff entre margem e erro, não foi configurado no experimento atual sendo o valor padrão ("C=0.01") do scikit learn adotado.

### k-NN

In [ ]:
# Define os valores a serem testados no hiperparametro
values  = [2, 5, 10, 20, 30, 40, 50, int(df.shape[0]/100), 100, 250, 300]
results = []

# Testa cada valor no hiperparametro e apresenta as metricas apuradas
for value in values:
    title = 'k Nearest Neighbor (n_neighbors={})'.format(value)
    knn = neighbors.KNeighborsClassifier(n_neighbors=value)
    knn = run_classifier(df=df, clf=knn)
    show_metrics(title, knn)
    
    results.append(knn)

# Demonstra graficamente os resultados do kNN
title  = 'k Nearest Neighbor'
xlabel = 'Number of neighbors'

plot_values(title, xlabel, values, results)

##### Análise dos resultados

Para o algoritmo __kNN__ o hiperparâmetro variado foi o número de vizinhos próximos a serem analisados, denotado por "k". Todas as tentativas de variação de "k" apresentaram _overfitting_ ou _underfitting_, sendo "k=2" o mais expressivo dentre aqueles exibiram _overfitting_.

O melhor modelo foi aquele com "k=40", exibindo diferença entre erros de 0.64. 

### Random Forest

In [ ]:
# Define os valores a serem testados no hiperparametro
values  = [2, 5, 10, 20, 30, 40, 50, int(df.shape[0]/100), 100, 250, 300]
results = []

# Testa cada valor no hiperparametro e apresenta as metricas apuradas
for value in values:
    title = 'Random Forest (n_estimators={})'.format(value)
    rf = ensemble.RandomForestClassifier(n_estimators=value)
    rf = run_classifier(df=df, clf=rf)
    show_metrics(title, rf)
    
    results.append(rf)

# Demonstra graficamente os resultados do Random Forest
title  = 'Random Forest'
xlabel = 'Number of trees'
plot_values(title, xlabel, values, results)

##### Análise dos resultados

O algoritmo de _bagging_ __random forest__ teve seu número de árvores, também chamadas de estimadores, variado. A técnica adotada pelo _bagging_ ataca o erro de variância, ao separar os dados de treino ($D$) em $i$ conjuntos, treinar $i$ modelos para cada $D_i$ partição e adotar a média das predições, metodologia esta muito similar a validação cruzada.

Ao variar o número de árvores, $i$ da explicação anterior, foi interessante observar que a partir de "n_estimators=40" as métricas do classificador variaram minimamente, indicando que tal valor é aquele que leva a convergência do método.

De forma similar as demais técnicas até então abordadas, principalmente a árvore de decisão, os modelos elucidados pelo classificador exibiram indícios de _overfitting_, ocorrendo empate de performance a partir de "n_estimators=40".

### Gradient Tree Boosting

In [ ]:
# Define os valores a serem testados no hiperparametro
values = [0.02, 0.05, 0.1, 2, 5, 10, 20, 30, 40, 50, int(df.shape[0]/100)]
for i in range(len(values)):
    values[i] = int(values[i] * 100)

results = []

# Testa cada valor no hiperparametro e apresenta as metricas apuradas
for value in values:
    title = 'Gradient Tree Boosting (n_estimators={})'.format(value)
    gtb = ensemble.GradientBoostingClassifier(n_estimators=value)
    gtb = run_classifier(df=df, clf=gtb)
    show_metrics(title, gtb)
    
    results.append(gtb)

# Demonstra graficamente os resultados da Gradient Tree Boosting
title  = 'Gradient Tree Boosting'
xlabel = 'Number of estimators'
plot_values(title, xlabel, values, results)

##### Análise dos resultados

Por fim, um algoritmo de _boosting_ participou do experimento. A metodologia do _boosting_, no geral, busca associar modelos simples para gerar um classificador bastante capacitado. O __gradient tree boosting__ opta por mesclar árvores de regressão, buscando a minimização de sua função de perda que, por sua vez, leva em conta o desvio dos exemplos.

Aqui, o número de árvores ("n_estimators") também foi alternado e todos os valores multiplicados por 100. Três novas possiblidades foram adicionadas com o intuito de analisar uma recomendação feita pela própria biblioteca de aprendizado que explicita que um número maior de estimadores tende a levar a uma melhor performance do algoritmo, devido a robustez a _overfitting_ do mesmo, sendo 100 o valor padrão aplicado pelo scikit.

A análise das métricas demonstrou que a partir de "n_estimators=10" o algoritmo já atinge sua convergência, tendo sua diferença entre erro empírico e erro estimado estabilizado. Além disso, quando o hiperparâmetro assumiu valor igual 10 o método apresentou seu melhor resultado de aproximação. 

Também foi possível observar que para um número baixo de estimadores o classificador de fato não performa tão bem chegando a predizer somente uma classe para todos os exemplos, contudo um número tão grande quanto 100 estimadores não se mostra significativamente necessário, levando em conta a base de dados em questão. 

Por fim, em conformidade ao apontado pela ferramenta, aqueles modelos que foram testados com maiores quantidades de estimadores apresentaram _overfitting_ controlado, não ultrapassando 0.03 pontos percentuais.

### Comparação dos Melhores Modelos

Para a comparação dos resultados, somente as __curvas roc__ dos melhores modelos serão aprensentadas, visto que os valores das métricas para cada um dos experimentos se encontram acima. 

Abaixo as curvas, para cada _fold_ da base de dados:

In [ ]:
# Apresenta a curva roc do melhor modelo Arvore de Decisao
plot_roc_curve('Decision Tree', tree.DecisionTreeClassifier(max_depth=5))

In [ ]:
# Apresenta a curva roc do melhor modelo SVM
plot_roc_curve('Support Vector Machine', svm.SVC(gamma='auto', 
                              kernel='linear', probability=True))

In [ ]:
# Apresenta a curva roc do melhor modelo KNN
plot_roc_curve('Random Forest', 
               ensemble.RandomForestClassifier(n_estimators=40))

In [ ]:
# Apresenta a curva roc do melhor modelo KNN
plot_roc_curve('k Nearest Neighbor', 
               neighbors.KNeighborsClassifier(n_neighbors=40))

In [ ]:
# Apresenta a curva roc do melhor modelo Gradient Tree Boosting
plot_roc_curve('Gradient Tree Boosting', 
               ensemble.GradientBoostingClassifier(n_estimators=10))

##### Análise dos resultados

A __curva roc__ é responsável por apresentar uma forma clara do quão bem o modelo se saiu em sua classifcação, utilizando as taxas de verdadeiro positivo, ou seja, exemplos que pertenciam a uma categoria e de fato foram associados a ela e taxa de falsos positivos, que define o caso contrário, onde o algoritmo errou sua predição. 

O quinto _fold_ foi aquele que demonstrou maior dificuldade para os modelos ainda que o mesmo possuísse a mesmas características que os demais. Para tal _fold_ algums modelos chegaram a apresentar curvas lineares com algumas inflexões de função degrau.

A partir da análise visual das curvas roc de cada classificador foi possível perceber que a __árvore de decisão__ obteve os melhores resultados, sendo o único classificador que não exibiu a curva do quinto _fold_ próxima a linha de orientação do gráfico.

Quanto as métricas, aquele modelos que apresentou melhor acurácia de treino e, por consequente, melhor erro empírico foi o:

In [ ]:
print(best_ein[0])

No que tange o erro esperado, o modelo que se sobressaiu foi o:

In [ ]:
print(best_eout[0])

Para a precisão nas classes "False Positive" o modelo 

In [ ]:
print(best_precision_false[0])

exibiu os mais apurados resultados. Já para a classe "Confirmed"

In [ ]:
print(best_precision_true[0])

se destacou.

Por fim, temos a métrica revocação, nesta, para a classe "False Positive" o modelo

In [ ]:
print(best_recall_false[0])

demonstrou os melhor resultados, enquanto o 

In [ ]:
print(best_recall_true[0])

se sobressaiu para a classe "Confirmed".